In [23]:
import pandas as pd
import spacy
from konlpy.tag import Mecab
import numpy as np

In [19]:
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

In [27]:
# 데이터 로드, 컬럼명 지정
df = pd.read_excel('./total.xlsx', header=0)
df.columns = ['idx', 'ko_original', 'en_original', 'matis']
df = df.dropna(subset=['matis']).reset_index(drop=True)

### KO BLEU
ref : http://incredible.ai/nlp/2020/02/29/BLEU/#korean

In [28]:
# bleu_score, Smoothing Function
cc = SmoothingFunction()

# Sentence splitter
mecab = Mecab()

In [29]:
for idx, row in df.iterrows():

    references = [mecab.morphs(row['ko_original'])]
    target_text = mecab.morphs(row['matis'])
    score_1gram = sentence_bleu(references, target_text, weights=[1,0,0,0], smoothing_function=cc.method2, auto_reweigh=True)
    score_2gram = sentence_bleu(references, target_text, weights=[0,1,0,0], smoothing_function=cc.method2, auto_reweigh=True)
    score_3gram = sentence_bleu(references, target_text, weights=[0,0,1,0], smoothing_function=cc.method2, auto_reweigh=True)
    score_4gram = sentence_bleu(references, target_text, weights=[0,0,0,1], smoothing_function=cc.method2, auto_reweigh=True)
    scorelist = [score_1gram, score_2gram, score_3gram, score_4gram]
    average_bleu = np.mean(scorelist)   
    df.loc[idx, 'matis_bleu'] = average_bleu

In [30]:
df

,idx,ko_original,en_original,matis,matis_bleu
0,conversation short : 1,내 동료들과 나는 이 프로젝트를 진행하고 있어.,My colleagues and I are working on this project.,내 동료들과 나는 이 프로젝트를 진행하고 있습니다.,0.875824
1,2,네가 한 말 당장 취소해.,Take back what you said right now.,당장 당신이 말한 것을 실행 취소.,0.209028
2,3,그 변호사는 의뢰인의 비밀을 지켰어.,The lawyer kept her client’s scret.,변호사는 고객의 비밀을 지켰다.,0.446088
3,4,나는 일찍 집에 가고 싶어.,I want to go home early,일찍 집에 가고 싶어.,0.778801
4,5,나는 내가 키가 더 컸으면 좋겠어.,I wish I were taller.,키가 더 컸으면 좋았을 텐데,0.352004
...,...,...,...,...,...
245,246,시장에선 가계 예금 중 일부라도 유입되면 올해 중국 증시가 활기를 띨 것이란 기대가...,Market watchers expect the Chinese stock marke...,시장 구경꾼들은 올해 가계 예금의 일부라도 도입되면 중국 증시가 활기를 띨 것으로 ...,0.326847
246,247,하지만 소프트뱅크 비전펀드에는 아랍계 자산가들의 투자 비중이 가장 높은 것으로 알려...,However Softbank Vision Fund is known to have ...,소프트뱅크 비전펀드는 부유한 아랍인의 투자 비중이 가장 높은 것으로 알려졌다.,0.499374
247,248,과열종목 지정 요건을 완화하고 공매도 금지기간을 연장해도 업틱룰 예외 조항을 손질하...,They argue that even if the requirements for d...,과열주식 지정 요건을 완화하고 공매도 금지를 연장하더라도 ‘업틱 규칙 예외조항’이 ...,0.365348
248,249,투자자가 만기 전에 조기상환청구권을 행사할 경우 발행사는 원금에 해당 기간까지의 약...,If an investor exercises his or her right to c...,투자자가 만기 전에 조기상환청구권을 행사하면 발행인은 원금과 약정이자를 그 기간까지...,0.406046


In [31]:
import numpy as np
bleulist = list(df["matis_bleu"])
average = np.mean(bleulist)
print(f"Average BLEU : {round(average,2)}")

Average BLEU : 0.38


In [32]:
# bleu score만 있는 text file 생성 (excel에 붙여넣을 용)

bleulist = list(df["matis_bleu"])

with open('bleulist.txt', 'w') as textfile:
    
    for bleu in bleulist:
        textfile.write(str(bleu)+"\n")
    
    textfile.write(str(round(average,2))+"\n")
        
textfile.close()

### EN BLEU

In [9]:
# Sentence Splitter
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe('sentencizer')

# bleu_score, Smoothing Function
cc = SmoothingFunction()

OSError: [E050] Can't find model 'ko_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [7]:
# MATIS BLEU_SCORE 생성
df['matis_bleu'] = ''
for idx, row in df.iterrows():
    references = [row['en_original'].split(' ')]
    target_text = row['matis'].split(' ')
    score = sentence_bleu(references, target_text, weights=[1, 0, 0, 0], smoothing_function=cc.method2)
    df.loc[idx, 'matis_bleu'] = score